In [1]:
import os
import copy
import time
import torch
import skimage
import random
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform
print("complete")

complete


In [2]:
train_vectors = np.load("train_vectors_air.npy")
test_vectors = np.load("test_vectors_air.npy")

In [3]:
def pro_select(vectors, probs, num):
    points = np.zeros((num, 2))
    core_point = np.zeros((1, 2))
    for i in range(num):
        x = random.uniform(0, 1)
        cumprob = 0.0
        k = -1
        while x > cumprob:
            k += 1
            cumprob += probs[k]
        points[i, 0], points[i, 1] = vectors[k, 0], vectors[k, 1]
        
        adj_list = []
        core_point[0, 0], core_point[0, 1] = vectors[k, 0], vectors[k, 1]
        dis = np.sum(pow(vectors - core_point, 2), axis=1)
        for j in range(len(dis)):
            if dis[j] <= 8:
                adj_list.append(j)
        for j in range(len(adj_list)):
            probs[adj_list[j]] /= 100.0
            
        probs[k] = 0
        probs = probs/np.sum(probs)
        
    return points

class Bird_Test_Process(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, train_file, root_dir, train_vectors, num, transform=None):
        f = open(train_file, 'r')
        self.train_list = f.readlines()
        f.close()
        self.root_dir = root_dir
        self.transform = transform
        self.train_vectors = train_vectors
        self.part_num = num

    def __len__(self):
        return len(self.train_list)

    def __getitem__(self, idx):
        line = self.train_list[idx]
        img_dir_label = line.strip('\n').split(' ')
        img_dir = os.path.join(self.root_dir, img_dir_label[0])
        image = io.imread(img_dir)
        if len(image.shape) == 2:
            image = image[:,:,np.newaxis]
            image = np.tile(image, [1, 1, 3])
            
        image = skimage.util.img_as_ubyte(transform.resize(image, (448, 448))) 
        label = int(img_dir_label[1])
        sample = {}
        
        vectors = self.train_vectors[idx]

        points = pro_select(vectors[:, :2].astype(int), copy.deepcopy(vectors[:, 3]), self.part_num)

        points1 = pro_select(vectors[:, :2].astype(int), copy.deepcopy(vectors[:, 3]), self.part_num)

        points2 = pro_select(vectors[:, :2].astype(int), copy.deepcopy(vectors[:, 3]), self.part_num)

        for i in range(self.part_num):
            x, y = points[i][0] * 16 + 8, points[i][1] * 16 + 8
            l = random.randint(46, 98)
            m1, m2, n1, n2 = int(np.maximum(0, x - l)), int(np.minimum(448, x + l)), \
                            int(np.maximum(0, y - l)), int(np.minimum(448, y + l))
            sample['image%d' % i] = image[m1:m2, n1:n2]
            x1, y1 = points1[i][0] * 16 + 8, points1[i][1] * 16 + 8
            l1 = random.randint(46, 98)
            m1, m2, n1, n2 = int(np.maximum(0, x1 - l1)), int(np.minimum(448, x1 + l1)), \
                            int(np.maximum(0, y1 - l1)), int(np.minimum(448, y1 + l1))
            sample['s_image%d' % i] = image[m1:m2, n1:n2]
            x2, y2 = points2[i][0] * 16 + 8, points2[i][1] * 16 + 8
            l2 = random.randint(46, 98)
            m1, m2, n1, n2 = int(np.maximum(0, x2 - l2)), int(np.minimum(448, x2 + l2)), \
                            int(np.maximum(0, y2 - l2)), int(np.minimum(448, y2 + l2))
            sample['t_image%d' % i] = image[m1:m2, n1:n2]
            
        sample['label'] = label

        if self.transform:
            for i in range(self.part_num):
                sample['image%d' % i] = self.transform(sample['image%d' % i])
                sample['s_image%d' % i] = self.transform(sample['s_image%d' % i])
                sample['t_image%d' % i] = self.transform(sample['t_image%d' % i])

        return sample
    
class Bird_Dataset_Process(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, train_file, root_dir, train_vectors, num, transform=None):
        f = open(train_file, 'r')
        self.train_list = f.readlines()
        f.close()
        self.root_dir = root_dir
        self.transform = transform
        self.train_vectors = train_vectors
        self.part_num = num

    def __len__(self):
        return len(self.train_list)

    def __getitem__(self, idx):
        line = self.train_list[idx]
        img_dir_label = line.strip('\n').split(' ')
        img_dir = os.path.join(self.root_dir, img_dir_label[0])
        image = io.imread(img_dir)
        if len(image.shape) == 2:
            image = image[:,:,np.newaxis]
            image = np.tile(image, [1, 1, 3])
            
        image = skimage.util.img_as_ubyte(transform.resize(image, (448, 448))) 
        label = int(img_dir_label[1])
        sample = {}
        
        vectors = self.train_vectors[idx]
        points = pro_select(vectors[:, :2].astype(int), vectors[:, 3], self.part_num)
        for i in range(self.part_num):
            x, y = points[i][0] * 16 + 8, points[i][1] * 16 + 8
            l = random.randint(46, 98)
            m1, m2, n1, n2 = int(np.maximum(0, x - l)), int(np.minimum(448, x + l)), \
                            int(np.maximum(0, y - l)), int(np.minimum(448, y + l))
            sample['image%d' % i] = image[m1:m2, n1:n2]
        
        sample['label'] = label

        if self.transform:
            for i in range(self.part_num):
                sample['image%d' % i] = self.transform(sample['image%d' % i])

        return sample

In [4]:
data_transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize(224),
                transforms.RandomCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.480, 0.511, 0.534], [0.221, 0.215, 0.246])
            ])

root_dir = '/home/xinsir/Pytorch/air_data/'
train_file = '/home/xinsir/Pytorch/air_data/train_list.txt'
test_file = '/home/xinsir/Pytorch/air_data/test_list.txt'

img_data = {'train': Bird_Dataset_Process(train_file, root_dir, train_vectors, 4, data_transform),
                  'test': Bird_Test_Process(test_file, root_dir, test_vectors, 4, data_transform)}
loader_img = {x: torch.utils.data.DataLoader(img_data[x], batch_size=20,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'test']}

dataset_sizes = {x: len(img_data[x]) for x in ['train', 'test']}
print(img_data['test'][0])

/home/xinsir/.tensorflow/lib/python3.5/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/xinsir/.tensorflow/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/home/xinsir/.tensorflow/lib/python3.5/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


{'image0': tensor([[[ 0.2413,  0.2591,  0.2768,  ...,  0.2946,  0.2768,  0.2591],
         [ 0.2413,  0.2591,  0.2946,  ...,  0.2946,  0.2768,  0.2591],
         [ 0.2591,  0.2768,  0.3123,  ...,  0.2768,  0.2768,  0.2768],
         ...,
         [ 0.5430,  0.5252,  0.5075,  ..., -0.4685, -0.3265, -0.1668],
         [ 0.6140,  0.5785,  0.5252,  ..., -0.3265, -0.3620, -0.3442],
         [ 0.6140,  0.5785,  0.5075,  ..., -0.2378, -0.3975, -0.5039]],

        [[ 0.2133,  0.2316,  0.2498,  ...,  0.2680,  0.2498,  0.2316],
         [ 0.2133,  0.2316,  0.2498,  ...,  0.2680,  0.2498,  0.2316],
         [ 0.2316,  0.2316,  0.2498,  ...,  0.2498,  0.2498,  0.2498],
         ...,
         [ 0.0127, -0.0056, -0.0238,  ..., -1.2094, -0.9358, -0.6804],
         [ 0.1768,  0.1039,  0.0127,  ..., -0.9540, -0.8993, -0.8446],
         [ 0.2133,  0.1221, -0.0056,  ..., -0.7534, -0.8993, -1.0088]],

        [[ 0.2842,  0.3002,  0.3161,  ...,  0.3321,  0.3161,  0.3002],
         [ 0.2842,  0.3002,  0.316

In [5]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())))
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            start_time = time.time()        
            if epoch % 5 == 0 and phase == 'train':
                for param_group in optimizer.param_groups:
                    lr = param_group['lr']
                    print("***********************")
                    print("learning rate = %f" % lr)
            
            # Iterate over data.
            for i_batch, sample_batched in enumerate(loader_img[phase]):
                img0 = sample_batched['image0']
                img1 = sample_batched['image1']
                img2 = sample_batched['image2']
                img3 = sample_batched['image3']
                labels = sample_batched['label']
                if phase == 'test':
                    s_img0 = sample_batched['s_image0']
                    s_img1 = sample_batched['s_image1']
                    s_img2 = sample_batched['s_image2']
                    s_img3 = sample_batched['s_image3']
                    t_img0 = sample_batched['t_image0']
                    t_img1 = sample_batched['t_image1']
                    t_img2 = sample_batched['t_image2']
                    t_img3 = sample_batched['t_image3']
                    s_img0 = s_img0.to(device)
                    s_img1 = s_img1.to(device)
                    s_img2 = s_img2.to(device)
                    s_img3 = s_img3.to(device)
                    t_img0 = t_img0.to(device)
                    t_img1 = t_img1.to(device)
                    t_img2 = t_img2.to(device)
                    t_img3 = t_img3.to(device)
                    
                img0 = img0.to(device)
                img1 = img1.to(device)
                img2 = img2.to(device)
                img3 = img3.to(device)
                labels = labels.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    if phase == 'train':
                        output0 = model(img0)
                        output1 = model(img1)
                        output2 = model(img2)
                        output3 = model(img3)
                    else:
                        output0 = (model(img0) + model(s_img0) + model(t_img0))/3.0
                        output1 = (model(img1) + model(s_img1) + model(t_img1))/3.0
                        output2 = (model(img2) + model(s_img2) + model(t_img2))/3.0
                        output3 = (model(img3) + model(s_img3) + model(t_img3))/3.0
                    outputs = output0 + output1 + output2 + output3
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * img3.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                if i_batch % 50 == 0 and phase == 'train':
                    print("Iteration %d, loss = %f" % (i_batch, loss))

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            if phase == 'test':
                scheduler.step(epoch_loss)
                
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            cost_time = (time.time() - start_time)/60.0
            print('{} 1 epoch time: {:.2f}min'.format(
                phase, cost_time))
            
            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [6]:
import time
import copy
import warnings
import torch.optim as optim
from torch.optim import lr_scheduler

warnings.filterwarnings('ignore')

device = torch.device("cuda:0")

vgg_16 = models.vgg16(pretrained=True)
vgg_16.classifier = nn.Sequential(*list(vgg_16.classifier.children())[:1])
vgg_16.features[30] = nn.AvgPool2d(kernel_size=14, stride=1, padding=0)
vgg_16.features.add_module('31', nn.Dropout(0.8)) 
vgg_16.classifier[0] = nn.Linear(in_features=512, out_features=200, bias=True)
# vgg_16 = nn.DataParallel(vgg_16, device_ids=[2, 3])
vgg_16 = vgg_16.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(vgg_16.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0005)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_ft, mode='min', factor=0.5, patience=5)

model = train_model(vgg_16, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=60)

Epoch 0/59
----------
2018-12-20 11:07:40
----------
***********************
learning rate = 0.001000
Iteration 0, loss = 5.717798
Iteration 50, loss = 4.911721
Iteration 100, loss = 5.235509
Iteration 150, loss = 4.664189
Iteration 200, loss = 4.412017
Iteration 250, loss = 4.332121
Iteration 300, loss = 4.143062
train Loss: 4.6055 Acc: 0.0484
train 1 epoch time: 4.29min
test Loss: 3.3975 Acc: 0.2112
test 1 epoch time: 2.03min

Epoch 1/59
----------
2018-12-20 11:13:59
----------
Iteration 0, loss = 4.366677
Iteration 50, loss = 3.301910
Iteration 100, loss = 3.926996
Iteration 150, loss = 3.249774
Iteration 200, loss = 3.098011
Iteration 250, loss = 2.521573
Iteration 300, loss = 3.138310
train Loss: 3.2568 Acc: 0.1927
train 1 epoch time: 4.38min
test Loss: 2.2387 Acc: 0.4167
test 1 epoch time: 2.03min

Epoch 2/59
----------
2018-12-20 11:20:23
----------
Iteration 0, loss = 2.833836
Iteration 50, loss = 2.525953
Iteration 100, loss = 3.044785
Iteration 150, loss = 2.587960
Iteration

train Loss: 0.5084 Acc: 0.8335
train 1 epoch time: 4.42min
test Loss: 0.6394 Acc: 0.8260
test 1 epoch time: 2.32min

Epoch 21/59
----------
2018-12-20 13:30:57
----------
Iteration 0, loss = 0.297842
Iteration 50, loss = 0.847117
Iteration 100, loss = 0.475477
Iteration 150, loss = 0.436345
Iteration 200, loss = 0.258736
Iteration 250, loss = 0.665388
Iteration 300, loss = 0.309002
train Loss: 0.4654 Acc: 0.8461
train 1 epoch time: 4.39min
test Loss: 0.7037 Acc: 0.7999
test 1 epoch time: 2.20min

Epoch 22/59
----------
2018-12-20 13:37:32
----------
Iteration 0, loss = 0.529705
Iteration 50, loss = 0.603974
Iteration 100, loss = 0.412978
Iteration 150, loss = 0.432946
Iteration 200, loss = 0.579581
Iteration 250, loss = 0.421997
Iteration 300, loss = 0.494491
train Loss: 0.4241 Acc: 0.8608
train 1 epoch time: 4.40min
test Loss: 0.6369 Acc: 0.8275
test 1 epoch time: 2.15min

Epoch 23/59
----------
2018-12-20 13:44:05
----------
Iteration 0, loss = 0.680923
Iteration 50, loss = 0.225938


Iteration 250, loss = 0.214584
Iteration 300, loss = 0.031060
train Loss: 0.1304 Acc: 0.9570
train 1 epoch time: 4.42min
test Loss: 0.6182 Acc: 0.8545
test 1 epoch time: 2.54min

Epoch 42/59
----------
2018-12-20 15:53:28
----------
Iteration 0, loss = 0.072307
Iteration 50, loss = 0.027461
Iteration 100, loss = 0.129040
Iteration 150, loss = 0.026460
Iteration 200, loss = 0.014527
Iteration 250, loss = 0.159611
Iteration 300, loss = 0.022015
train Loss: 0.0963 Acc: 0.9673
train 1 epoch time: 4.44min
test Loss: 0.6394 Acc: 0.8707
test 1 epoch time: 2.53min

Epoch 43/59
----------
2018-12-20 16:00:26
----------
Iteration 0, loss = 0.052784
Iteration 50, loss = 0.134946
Iteration 100, loss = 0.024862
Iteration 150, loss = 0.148849
Iteration 200, loss = 0.008731
Iteration 250, loss = 0.122125
Iteration 300, loss = 0.105248
train Loss: 0.1006 Acc: 0.9679
train 1 epoch time: 4.47min
test Loss: 0.6231 Acc: 0.8635
test 1 epoch time: 2.34min

Epoch 44/59
----------
2018-12-20 16:07:14
--------

In [8]:
torch.save(model, '/home/xinsir/Pytorch/random_part_air')